In [16]:
import json
import minsearch
from openai import OpenAI

In [23]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [24]:
def search(query):

    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [19]:
def build_prompt(query, search_results):

    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the Q&A database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}

    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(
        question=query,
        context=context
    ).strip()

    return prompt

In [28]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag('The course has staretd. Can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for the final projects. Avoid leaving everything until the last minute.'